In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tarfile
import re

In [9]:
import os

# Replace the path with your actual path
visual_studio_path = r'C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.38.33130\bin\Hostx64\x64'

# Add the path to the PATH environment variable
os.environ['PATH'] += ';' + visual_studio_path

# Check if the path is now in the PATH environment variable
if visual_studio_path in os.environ['PATH']:
    print(f"{visual_studio_path} is now in the PATH.")
else:
    print(f"Failed to add {visual_studio_path} to the PATH.")


C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.38.33130\bin\Hostx64\x64 is now in the PATH.


In [18]:
import os
print(os.environ['C:\Users\harri\Downloads\fasttext-0.9.2.tar.gz\fasttext-0.9.2\src'])


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (894436919.py, line 2)

In [80]:
headers = ['lang', 'text']

df = pd.read_csv("europarl.csv", header=None, names=headers)
#df['text'] = re.sub(r'<.*?>', '', in_text).lower().strip()
df['text'] = df['text'].astype(str)
df['text'] = df['text'].apply(lambda x: re.sub(r'<.*?>', '', x).lower().strip())
df

,lang,text
0,bg,състав на парламента: вж. протоколи
1,bg,одобряване на протокола от предишното заседани...
2,bg,състав на парламента: вж. протоколи
3,bg,проверка на пълномощията: вж. протоколи
4,bg,внасяне на документи: вж. протоколи
...,...,...
832300,sv,lannoye har just lagt fram begäran från sin gr...
832301,sv,vill någon yttra sig för denna begäran?
832302,sv,. (de) fru talman! som vi redan vid upprepade ...
832303,sv,"vi skall nu lyssna på föredragande, fru klaß, ..."


In [82]:
sampled_df = df.groupby('lang', group_keys=False).apply(lambda x: x.sample(min(len(x), 40000)))
print(sampled_df.shape)

(832305, 2)


In [85]:
df = sampled_df.reindex(np.random.permutation(sampled_df.index)).reset_index(drop=True)
df[:3]

,lang,text
0,de,"abschließend sagt die rahmenrichtlinie auch, d..."
1,it,"non dimentichiamo però una cosa, malgrado tutt..."
2,nl,"mijnheer de voorzitter, geachte collega's, de ..."


In [86]:
def normalize_text(row):
    
    label = '__label__' + str(row['lang'])
    txt = str(row['text'])
    
    return ' '.join(( label + ' , ' + txt ).split())

In [87]:
df['normalized'] = df.apply( lambda row: normalize_text(row), axis=1 )

In [88]:
SPLIT = 630000
train = df['normalized'][:SPLIT].copy()
test = df['normalized'][SPLIT:].copy()

In [95]:
np.savetxt('europarl_train.txt', train.values, fmt='%s', encoding='utf-8')
np.savetxt('europarl_eval.txt', test.values, fmt='%s', encoding='utf-8')